In [1]:
# Custom imports
from Generate_Problem import generate_problem
from HHL_Circuit import hhl_circuit
from Iterative_Refinement import norm_estimation, sign_estimation, IR
from Quantum_Linear_Solver import quantum_linear_solver

# Third-party libraries
import numpy as np
import pandas as pd

# Qiskit imports
import qiskit
import qiskit_aer

# PyTKet imports
import pytket
from pytket.extensions.nexus import NexusBackend
from pytket.extensions.nexus import QuantinuumConfig


# Filter warnings
# warnings.filterwarnings("ignore")

# Matplotlib inline to visualize plots
%matplotlib inline

# Use py310-qiskit1.0 environment

# Print library versions
print("Qiskit version (should be 1.3.0):", qiskit.__version__)
print("Qiskit Aer version (should be 0.15.1):", qiskit_aer.__version__)
print("Pytket version (should be 1.36.0):", pytket.__version__)

Qiskit version (should be 1.3.0): 1.2.2
Qiskit Aer version (should be 0.15.1): 0.15.1
Pytket version (should be 1.36.0): 1.39.0


In [2]:
problem = generate_problem(4, cond_number=5, sparsity=0.5, seed=1)
problem

{'A': array([[ 0.24, -0.  ,  0.  , -0.03],
        [-0.  ,  0.61,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.51, -0.  ],
        [-0.03,  0.  , -0.  ,  0.56]]),
 'b': array([ 0.26, -0.1 , -0.08, -0.17]),
 'csol': array([ 1.05243446, -0.16393443, -0.15686275, -0.24719101]),
 'condition_number': 2.571541615252496,
 'sparsity': 0.625,
 'eigs': array([0.23721179, 0.51      , 0.56278821, 0.61      ])}

In [3]:
# Create config object to specify details of desired backend
configuration = QuantinuumConfig(device_name='H2-2')

# Set up a Nexus Project to compartmentalize running jobs and data
# Create a NexusBackend using our config and our retrieved project
backend = NexusBackend(configuration, project_name='HHL-IR')


Started using project with name: HHL-IR


In [4]:
A = problem['A']
b = problem['b']

# Single Run
solution = quantum_linear_solver(A, b, backend=backend, t0=2*np.pi, shots=1024)
solution

Running on H2-2


{'cost': 0,
 'number_of_qubits': 7,
 'circuit_depth': 330,
 'total_gates': 517,
 'two_qubit_gates': 167,
 'result_handle': ResultHandle('929cca77-68f2-4dc3-a452-af59f6b5c94f', 3009234),
 'runtime': datetime.timedelta(seconds=30, microseconds=765886),
 'x': array([ 0.35985609, -0.74428277, -0.28971167, -0.48230064]),
 'two_norm_error': 0.9430770753004064,
 'residual_error': 0.4093216791090782}

In [ ]:
status = backend.circuit_status(solution['result_handle'])
status

In [ ]:
# Iterative Refinement
refined_solution = IR(A, b, precision=1e-5, max_iter=5, backend=backend, plot=True)
refined_solution

Iteration: 1
Running on H2-2
residual: 0.24586805256739347
error: 0.9258937091462371
alpha: 0.4044523635877364

Iteration: 2
Running on H2-2
residual: 0.19756100373099197
error: 0.8245702262395642
alpha: 0.5284672709358116

Iteration: 3
Running on H2-2


In [ ]:
df = pd.DataFrame()
# Make a dataframe out of the results
datarow = {
    "Backend": backend.backend_config.device_name,
    "Problem Size": f"{len(b)} x {len(b)}",
    "A": problem["A"],
    "b": problem["b"],
    "Condition Number": problem["condition_number"],
    "Sparsity": problem["sparsity"],
    "Number of Qubits": solution["number_of_qubits"],
    "Circuit Depth": solution["circuit_depth"],
    "Total Gates": solution["total_gates"],
    "Two-Qubit Gates": solution["two_qubit_gates"],  # Uncomment if needed
    # "Runtime": solution["runtime"],  # Uncomment if needed
    "||x_c - x_q|| without IR": solution["two_norm_error"],
    "||x_c - x_q|| with IR": refined_solution["errors"][-1],
    "||Ax - b|| without IR": solution["residual_error"],
    "||Ax - b|| with IR": refined_solution["residuals"][-1],
    "Total Iterations of IR": refined_solution["total_iterations"],
    # "Total Cost": refined_solution["total_cost"],
    "Error list": refined_solution["errors"],
    "Residual list": refined_solution["residuals"],
}
df = pd.concat([df, pd.DataFrame([datarow])], ignore_index=True)

# Display the DataFrame without showing "Error list" and "Residual list"
df_display = df.drop(columns=["Error list", "Residual list", "A", "b"])

df_display.style.hide(axis="index").format(precision=6).set_caption(f"{backend.backend_config.device_name} Results")